## Python environment

With python3.9 the following commands in the command line should create an environment sufficient to run this notebook.

```
    envname="Qmin-vis-env"   
    python3 -m venv "$envname"
    source "$envname"/bin/activate  
    python3 -m pip install pandas pyvistaqt jupyterlab PyQt5 tqdm
    python3 -m ipykernel install --name="$envname" --user
```    

(This assumes `python3` points to an installation of python3.9. I'm not sure about other versions yet but it looks like python3.10 is not working as of 10/27/21. You may have to open jupyter lab as `python3 -m jupyter lab`.)

In [1]:
from pandas import read_csv
import pyvista as pv
import pyvistaqt as pvqt
import numpy as np

In [2]:
settings = {
    "boundaries_color":"green",
    "director_color":"red",
    "director_resolution":0.025, # "tolerance" for density of director glyphs (lower means more rods)
    "default_defect_S":0.3, # initialization order value for defect isosurfaces
    "defect_color":(37/256,150/256,190/256),
    "checkbox_size":50, # size of toggle boxes in pixels
    "checkbox_spacing":10, # spacing between toggle boxes in pixels
    "window_size":(1200,800), # window size in pixels
    "cylinder_resolution":8, # angular resolution for each cylindrical rod; larger values look nicer but take longer to compute
    "slice_plane_color":"lightyellow", # set to None to use slice_color_function instead    
    "slice_cmap":"cividis", # color map for use with slice_color_function
    "slice_color_function":(lambda slc: np.abs(slc["director"][:,0])), # optionally color slice plane by some function of director or order
    "plane_widget_color":"orange",
    "energy_L1_color":"yellow",
    "energy_L2_color":"pink",
    "energy_L3_color":"purple",
}

In [4]:
"""Get that file!"""

!scp dbeller@skyrmion.ucmerced.edu:open-Qmin-clones/clone7/tetstGUIbdys_x0y0z0.txt ./
dat = np.array(read_csv("tetstGUIbdys_x0y0z0.txt", sep="\t", header=None))

tetstGUIbdys_x0y0z0.txt                       100% 8275KB   1.7MB/s   00:04    


In [5]:
"""Parse the data"""

def Q33_from_Q5(Q5):
    (Qxx, Qxy, Qxz, Qyy, Qyz) = Q5.T
    Qmat = np.moveaxis(np.array([ 
            [Qxx, Qxy, Qxz],
            [Qxy, Qyy, Qyz],
            [Qxz, Qyz, -Qxx-Qyy]
            ]), -1, 0)
    return Qmat

def n_from_Q(Qmat):
    """Get director from 3x3-matrix Q-tensor data"""
    evals, evecs = np.linalg.eigh(Qmat)    
    return evecs[:,:,2]    
        
# name the data columns:
coords = dat[:,:3]
Qdata = dat[:,3:8]
Q33 = Q33_from_Q5(Qdata) # 3x3 Q matrix
site_types = dat[:,8]
order = dat[:,9]

# grid for pyvista:
Lx, Ly, Lz = [ int(item+1) for item in dat[-1,:3] ]
fullmesh = pv.UniformGrid((Lx,Ly,Lz)) 
# Order (defects) data:
order[site_types>0] = np.max(order) # Don't plot defects inside objects
fullmesh["order"] = order

# director data:
Qdata[site_types>0] = 0. # Don't bother calculating eigenvectors inside objects
fullmesh["director"] = n_from_Q(Q33)

# boundaries:
fullmesh["nematic_sites"] = 1*(site_types<=0)



In [6]:
Q33_xyz = Q33.reshape((Lx,Ly,Lz,3,3))
diQjk = np.moveaxis(
    np.array(
        [np.roll(Q33_xyz,-1, axis=i) - np.roll(Q33_xyz,1, axis=i) for i in range(3)]
    ),
    0, -3
)

# energy:
fullmesh["energy_L1"] = np.sum(diQjk**2, axis=(-1,-2,-3)).flatten()
fullmesh["energy_L2"] = np.sum((np.einsum("...iij->...j", diQjk))**2, axis=-1).flatten() # djQij dkQik = djQji dkQki = sum_i[(djQji)^2]
fullmesh["energy_L3"] = np.einsum("...ij,...ikl,...jkl", Q33_xyz, diQjk, diQjk).flatten()
fullmesh["energy_L24"] = np.einsum("...ijk,...kij", diQjk, diQjk).flatten() - fullmesh["energy_L2"] # diQjk dkQij - diQij dkQjk 
for i in [1,2,3,24]:
    fullmesh[f"energy_L{i}"] *= 1*(site_types==0)

In [7]:
"""Visualization routine"""

## global options
pv.global_theme.title = "open-Qmin visualization with pyvistaqt"

def update_Qmin_plot(pl):
    """Create/update the visualization with pyvista"""

    pl.enable_eye_dome_lighting()    
    pl.renderer.add_axes(interactive=True, color='black') # xyz axes arrows
    pl.renderer.set_background("white")

    def director_slice_func(normal, origin):
        """make glyph plot and transparent plane for director field slice"""
        slc = fullmesh.slice(normal=normal, origin=origin)
        cylinders = slc.glyph(orient="director", scale="nematic_sites",
            factor=Lx*settings["director_resolution"],
            geom=pv.Cylinder(
                radius=0.2, height=1, 
                resolution=settings["cylinder_resolution"]
            ),  
            tolerance=settings["director_resolution"], 
            progress_bar=True
        )
        pl.renderer.actors["director"] = pl.add_mesh(
            cylinders, 
            color=settings["director_color"], 
#             specular=1, ambient=0.35, 
            pbr=True, metallic=0.5, roughness=0.25, diffuse=1,
            name="director" # "name" == actor's name so old actor is replaced
        )
        pl.renderer.actors["slice_plane"] = pl.add_mesh(
            slc, opacity=0.01, 
            ambient=1, diffuse=0, specular=0, # glows, doesn't reflect
            color=settings["slice_plane_color"], 
            scalars=(settings["slice_color_function"](slc) 
                if settings["slice_plane_color"] is None 
                else None # use slice_color_function only if slice_plane_color==None
            ),
            cmap=settings["slice_cmap"],
            name="slice_plane" # "name" == actor's name so old actor is replaced
        )
        
    def generic_slider_callback(actor, scalars, contour_value, color=None, mesh_color_scalars=None, clim=None, cmap=None):
        """generic callback function for slider controlling isosurfaces"""
        the_contour = fullmesh.contour(
            [contour_value], scalars=scalars
        )
        kwargs={
            "show_scalar_bar":False,
            "specular":1, "specular_power":200,
            "ambient":0.5, "diffuse":1,
            "smooth_shading":True,
            "name":actor,
            "pbr":True, 
            "metallic":0,
            "roughness":0.25,
            "diffuse":1,
        }
        if mesh_color_scalars is not None:
            kwargs["scalars"] = mesh_color_scalars
            if clim is not None:
                kwargs["clim"] = clim
            if cmap is not None:
                kwargs["cmap"] = cmap            
        elif color is not None:
            kwargs["color"] = color
                
        pl.renderer.actors[actor] = pl.add_mesh(
            the_contour, 
            **kwargs
        )
    
    def update_defects(dthresh):
        return generic_slider_callback(
            "defects", "order", dthresh, color=None, mesh_color_scalars="energy_L24", clim=[-1,1], cmap="jet"
        )
    
    def update_Li(Li_value, i):
        return generic_slider_callback(
            f"L{i}_isosurface", f"energy_L{i}", Li_value, settings[f"energy_L{i}_color"]
        )
    

    pl.add_mesh(
        fullmesh.contour([0.5], scalars="nematic_sites"), 
        show_scalar_bar=False, 
        color=settings["boundaries_color"],
        name="boundaries",
        ambient=1,
#         specular=1, ambient=1, # lighting
#         smooth_shading=True, # very smooth! 
        pbr=True, metallic=0.5, roughness=0.25, diffuse=1
        )    
    
    ## the toggle buttons 
    
    def generic_toggle_vis(actor_name, flag):
        """utility for general toggle button action"""
        pl.renderer.actors[actor_name].SetVisibility(flag)
        
    def plane_widget_toggle(_, flag):
        if flag:
            pl.add_plane_widget(director_slice_func, 
                factor=1.1, 
                color=settings["plane_widget_color"],
                tubing=False
            )    
        else:
            pl.clear_plane_widgets()            
    plane_widget_toggle(_,True)   
    
    button_reference_x = settings["checkbox_spacing"] #settings["window_size"][0] - 4*settings["checkbox_spacing"] - settings["checkbox_size"]
    button_reference_y = settings["window_size"][1] - 3*settings["checkbox_size"] # settings["checkbox_spacing"]
    def button_coords(button_number, x_offset=0):
        """utility for placement of several buttons"""
        button_repeat_length = settings["checkbox_size"]+settings["checkbox_spacing"]        
        return [button_reference_x + x_offset*button_repeat_length, 
                button_reference_y - button_number*button_repeat_length]
    
    def add_toggle_button(actor, button_color, button_index, func=generic_toggle_vis, value=True):
        pl.add_checkbox_button_widget(
            lambda flag: func(actor, flag), 
            value=value, 
            color_on=button_color, background_color=button_color,
            color_off="white",
            position=button_coords(button_index)
        )
        
    add_toggle_button("defects", settings["defect_color"], 0)
    add_toggle_button("boundaries", settings["boundaries_color"], 1)
    add_toggle_button("director", settings["director_color"], 2)
    add_toggle_button("slice_plane", settings["slice_plane_color"], 3)  
    add_toggle_button(_, settings["plane_widget_color"], 4, func=plane_widget_toggle)      
    add_toggle_button("L1_isosurface", settings["energy_L1_color"], 5, value=False)
    add_toggle_button("L2_isosurface", settings["energy_L2_color"], 6, value=False)
    add_toggle_button("L3_isosurface", settings["energy_L3_color"], 7, value=False)
    
    ## the sliders
   
    def generic_isosurface_slider(callback_function, scalars, pointa, pointb, init_value=None, title=None, color=None):
        min_val = 1.01*np.min(fullmesh[scalars])
        max_val = 0.99*np.max(fullmesh[scalars])
        if min_val > max_val:
            tmp = min_val
            min_val = max_val
            max_val = tmp
        if init_value is None:
            init_value = 0.5*(min_val+max_val)      
        if color is not None:
            pv.global_theme.slider_styles.modern.slider_color = color
        pl.add_slider_widget(
            callback_function,
            [min_val, max_val], # value range
            value=init_value,
            color="black", # text color
            fmt="%0.3f", # text format
            pointa=pointa,
            pointb=pointb,
            style="modern",
            title=title
        )

    
    ref_pt = [ button_coords(-1.5, x_offset=1)[i]/settings["window_size"][i] for i in range(2) ] 
    
    defect_slider = generic_isosurface_slider(
        update_defects,
        "order",
        ref_pt, [ref_pt[0]+0.15, ref_pt[1]],
        init_value=0.3,
        color=settings["defect_color"]
    )
    
    for i in range(1,4):
        slider_y = 1. - 0.2*i
        generic_isosurface_slider(
            lambda value: update_Li(value, i),
            f"energy_L{i}",
            [0.05, slider_y], [0.15, slider_y],
            color=settings[f"energy_L{i}_color"],
            title=f"L{i}"
        )
        pl.renderer.actors[f"L{i}_isosurface"].SetVisibility(0)
    fullmesh.set_active_scalars="order"
    
    
    # Add a drop down menu

    def generic_menu_toggle(actor_name):
        actor=Qmin_plot.renderer.actors[actor_name]    
        def return_function():
            is_visible = actor.GetVisibility()
            actor.SetVisibility(1-is_visible)
        return return_function


    toggle_menu = Qmin_plot.main_menu.addMenu('Toggle')
    for actor_name in Qmin_plot.renderer.actors:
        toggle_menu.addAction(actor_name, 
                              generic_menu_toggle(actor_name))

    return pl


In [8]:
## create and configure the plotting window

pv.close_all()

Qmin_plot = pvqt.BackgroundPlotter(
    window_size=settings["window_size"], 
    #lighting='three lights'
)
Qmin_plot.add_mesh(fullmesh.outline(), color='black') # bounding box

update_Qmin_plot(Qmin_plot)

Cleaning: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:03<00:00]
Computing Glyphs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:00<00:00]


In [227]:
dir(Qmin_plot.renderer)

['AddActor',
 'AddActor2D',
 'AddCuller',
 'AddLight',
 'AddObserver',
 'AddViewProp',
 'AddVolume',
 'AutomaticLightCreationOff',
 'AutomaticLightCreationOn',
 'BackingStoreOff',
 'BackingStoreOn',
 'BreakOnError',
 'CAMERA_STR_ATTR_MAP',
 'CaptureGL2PSSpecialProp',
 'Clear',
 'ClearLights',
 'ComputeAspect',
 'ComputeVisiblePropBounds',
 'CreateLight',
 'DebugOff',
 'DebugOn',
 'DeviceRender',
 'DeviceRenderOpaqueGeometry',
 'DeviceRenderTranslucentPolygonalGeometry',
 'DisplayToLocalDisplay',
 'DisplayToNormalizedDisplay',
 'DisplayToView',
 'DisplayToWorld',
 'DrawOff',
 'DrawOn',
 'EraseOff',
 'EraseOn',
 'FastDelete',
 'GetActiveCamera',
 'GetActors',
 'GetActors2D',
 'GetAddressAsString',
 'GetAllocatedRenderTime',
 'GetAmbient',
 'GetAspect',
 'GetAutomaticLightCreation',
 'GetBackground',
 'GetBackground2',
 'GetBackgroundAlpha',
 'GetBackgroundAlphaMaxValue',
 'GetBackgroundAlphaMinValue',
 'GetBackgroundTexture',
 'GetBackingStore',
 'GetCenter',
 'GetClassName',
 'GetClippi

In [172]:
# Add a toolbar
from PyQt5.QtWidgets import  QAction

def add_action(toolbar, key, method, main_window):
    action = QAction(key, main_window)
    action.triggered.connect(method)
    toolbar.addAction(action)
    return

user_toolbar = Qmin_plot.app_window.addToolBar('User Toolbar')
add_action(user_toolbar, 'Line Widget', Qmin_plot.add_line_widget, Qmin_plot.app_window)

In [162]:
Qmin_plot.renderer.actors["defects"].SetVisibility(1)

In [174]:
dir(Qmin_plot)

['CreateTimer',
 'CursorChangedEvent',
 'DestroyTimer',
 'DrawChildren',
 'DrawWindowBackground',
 'Finalize',
 'GetRenderWindow',
 'HideCursor',
 'ICON_TIME_STEP',
 'IgnoreMask',
 'PaintDeviceMetric',
 'PdmDepth',
 'PdmDevicePixelRatio',
 'PdmDevicePixelRatioScaled',
 'PdmDpiX',
 'PdmDpiY',
 'PdmHeight',
 'PdmHeightMM',
 'PdmNumColors',
 'PdmPhysicalDpiX',
 'PdmPhysicalDpiY',
 'PdmWidth',
 'PdmWidthMM',
 'Render',
 'RenderFlag',
 'RenderFlags',
 'ShowCursor',
 'TimerEvent',
 '_ActiveButton',
 '_BasePlotter__before_close_callback',
 '_CURSOR_MAP',
 '_GetCtrlShift',
 '_GetKeyCharAndKeySym',
 '_Iren',
 '_QVTKRenderWindowInteractor__saveButtons',
 '_QVTKRenderWindowInteractor__saveModifiers',
 '_QVTKRenderWindowInteractor__saveX',
 '_QVTKRenderWindowInteractor__saveY',
 '_QVTKRenderWindowInteractor__wheelDelta',
 '_RenderWindow',
 '_Timer',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__

In [72]:
pv.close_all()

True

In [61]:
Qmin_plot.renderer.axes_widget.
Qmin_plot.renderer.axes_widget.SetPriority(0)

In [69]:
Qmin_plot.renderer.axes_actor.GetPosition()

(0.5, 10.0, 0.5)

In [115]:
dir(Qmin_plot)

['CreateTimer',
 'CursorChangedEvent',
 'DestroyTimer',
 'DrawChildren',
 'DrawWindowBackground',
 'Finalize',
 'GetRenderWindow',
 'HideCursor',
 'ICON_TIME_STEP',
 'IgnoreMask',
 'PaintDeviceMetric',
 'PdmDepth',
 'PdmDevicePixelRatio',
 'PdmDevicePixelRatioScaled',
 'PdmDpiX',
 'PdmDpiY',
 'PdmHeight',
 'PdmHeightMM',
 'PdmNumColors',
 'PdmPhysicalDpiX',
 'PdmPhysicalDpiY',
 'PdmWidth',
 'PdmWidthMM',
 'Render',
 'RenderFlag',
 'RenderFlags',
 'ShowCursor',
 'TimerEvent',
 '_ActiveButton',
 '_BasePlotter__before_close_callback',
 '_CURSOR_MAP',
 '_GetCtrlShift',
 '_GetKeyCharAndKeySym',
 '_Iren',
 '_QVTKRenderWindowInteractor__saveButtons',
 '_QVTKRenderWindowInteractor__saveModifiers',
 '_QVTKRenderWindowInteractor__saveX',
 '_QVTKRenderWindowInteractor__saveY',
 '_QVTKRenderWindowInteractor__wheelDelta',
 '_RenderWindow',
 '_Timer',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__